# Show round sizes for various margins at various quantiles
Compare with Table 1 from [BRAVO](https://www.usenix.org/system/files/conference/evtwote12/evtwote12-final27.pdf).

## Setup and define some utilities

In [1]:
from athena.audit import Audit
import pandas as pd

In [2]:
def athena_sample_sizes(
        risk_limit: float,
        p_w: float,
        p_r: float,
        sample_w: int,
        sample_r: int,
        p_completion: float,
        ballots_cast: int=100000
) -> int:
    """
    Return Athena round size based on completion probability, assuming the election outcome is correct.
    TODO: refactor to pass in integer vote shares to allow more exact calculations, and handle
    sampling without replacement.

    Inputs:
        risk_limit      - the risk-limit for this audit
        p_w             - the fraction of vote share for the winner
        p_r             - the fraction of vote share for the loser
        sample_w        - the number of votes for the winner that have already
                          been sampled
        sample_r        - the number of votes for the runner-up that have
                          already been sampled
        p_completion    - the desired chance of completion in one round,
                          if the outcome is correct

    Outputs:
        sample_size     - the expected sample size for the given chance
                          of completion in one round

    >>> athena_sample_sizes(0.1, 0.6, 0.4, 56, 56, 0.7)
    244
    """

    # calculate the undiluted "two-way" share of votes for the winner
    p_wr = p_w + p_r
    p_w2 = p_w / p_wr

    a = int(ballots_cast * p_w2)
    b = ballots_cast - a
    pstop_goal = [p_completion]
    if sample_w or sample_r:
        round_sizes = [sample_w + sample_r]
    else:
        round_sizes = []
    election = {
        "alpha": risk_limit,
        "delta": 1.0,
        "candidates": ["A", "B"],
        "results": [a, b],
        "ballots_cast": ballots_cast,
        "winners": 1,
        "name": 'pure_pair',
        "model": 'bin',
        "pstop_goal": pstop_goal,
    }

    a = Audit("athena", election['alpha'], election['delta'])
    a.add_election(election)
    a.add_round_schedule(round_sizes)
    if round_sizes:
        r = a.find_risk([sample_w])
        below_kmin = max(r['required']) - max(r['observed'])
    else:
         below_kmin = 0
    x = a.find_next_round_size(pstop_goal)
    next_round_size_0 = x['future_round_sizes'][0]

    next_round_size = next_round_size_0 + 2 * below_kmin

    return next_round_size

In [3]:
def ss(m, p_completion=0.9, risk_limit=0.1, ballots_cast=100000):
    return athena_sample_sizes(risk_limit, 0.5+m/2, 0.5-m/2, 0, 0, p_completion, ballots_cast)

In [4]:
margins = [.4, .3, .2, .16, .1, 0.08, 0.06, 0.04, 0.02, 0.01]
pstops = [0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

With 1,000,000 ballots

In [5]:
tab1m = {pstop: {m: ss(m, pstop, ballots_cast=1000000) for m in margins} for pstop in pstops}

In [6]:
pd.DataFrame(tab1m)

,0.25,0.50,0.75,0.90,0.95,0.99
0.40,14,18,31,42,58,183
0.30,22,30,52,77,114,229
0.20,44,75,122,184,274,732
0.16,72,120,190,282,427,854
0.10,182,290,484,710,1098,2930
0.08,278,429,720,1111,1830,3418
0.06,493,786,1258,1976,3418,6836
0.04,1088,1719,2832,4450,6836,13672
0.02,4248,6880,11331,17807,27344,54688
0.01,16796,27522,45330,71234,109375,218750


Note that results vary by number of ballots, for some reason

In [7]:
ss(0.02, p_completion=0.95, ballots_cast=1000000)

27344

In [8]:
ss(0.02, p_completion=0.95, ballots_cast=100000)

28125

With 100000 ballots

In [9]:
tab100k = {pstop: {m: ss(m, pstop, ballots_cast=100000) for m in margins} for pstop in pstops}

In [10]:
pd.DataFrame(tab100k)

,0.25,0.50,0.75,0.90,0.95,0.99
0.40,14,18,31,42,72,146
0.30,22,30,52,77,122,292
0.20,44,75,122,184,292,586
0.16,72,120,190,282,439,1172
0.10,182,290,484,710,1172,2344
0.08,278,429,720,1111,1758,4688
0.06,493,786,1258,1976,3028,6055
0.04,1088,1719,2832,4450,7031,18750
0.02,4248,6880,11331,17807,28125,75000
0.01,16796,27522,45330,71234,100000,100000


For testing....

In [11]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=1)

In [12]:
import logging
#logging.basicConfig(level="DEBUG")
# logging.basicConfig(level="WARNING")
logging.getLogger().setLevel("WARNING")
#logging.getLogger(__name__).setLevel("DEBUG")

In [13]:
logging.getLogger()

<RootLogger root (WARNING)>

In [14]:
logging.getLogger(__name__)

<Logger __main__ (WARNING)>